In [5]:
import pandas as pd
import tensorflow as tf

  # Cargar el dataset
data = pd.read_csv('data\dataset_wide.csv')

  # Mostrar las primeras filas del dataset
print(data.head())

          P         D          L          R P_fuzzy_set  P_membership  \
0  0.045553  0.608390 -18.517306  67.099168          ok      0.886118   
1 -0.979829  0.468399  62.832204  -7.498536   muy lejos      0.966382   
2  0.968055  0.035674 -34.584612 -12.701870   muy cerca      0.946758   
3  0.402764  0.032081 -71.360223  77.859054       cerca      0.995394   
4 -0.503925  0.629578  61.566158  16.442472       lejos      0.826791   

   D_fuzzy_set  D_membership                  L_fuzzy_set  L_membership  \
0  acercandose      1.000000        muy lento hacia atrás      0.574135   
1  acercandose      1.000000    rápidamente hacia delante      0.858390   
2   sin cambio      0.821631        despacito hacia atrás      0.541539   
3   sin cambio      0.839593  bastante rápido hacia atrás      0.784006   
4  acercandose      1.000000    rápidamente hacia delante      0.921692   

                     R_fuzzy_set  R_membership  \
0      rápidamente hacia delante      0.645042   
1         

In [10]:
#columnas a discretizar
columns_discret = ['P',
 'D',
 'L',
 'R',
 'P_membership',
 'D_membership',
 'L_membership',
 'R_membership'
 ]

data_discretized = pd.DataFrame()

In [11]:
for col in columns_discret:
    data_discretized[col+'_discretized'] = pd.cut(data[col], 256, labels=False)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(y)
num_decoder_tokens = len(tokenizer.word_index) + 1
y_tokenized = tokenizer.texts_to_sequences(y)

In [ ]:
max_len = max(len(s) for s in y_tokenized)

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_tokenized, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ajustar la longitud de las secuencias de frases en y_train a max_len
y_train = pad_sequences(y_train, maxlen=num_decoder_tokens, padding='post', truncating='post')

# Ajustar la longitud de las secuencias de frases en y_val a max_len
y_val = pad_sequences(y_val, maxlen=num_decoder_tokens, padding='post', truncating='post')

# Ajustar la longitud de las secuencias de frases en y_test a max_len
y_test = pad_sequences(y_test, maxlen=num_decoder_tokens, padding='post', truncating='post')

In [ ]:
print('tamaño X train', X_train.shape)
print('\ntamaño y train', y_train.shape)
print('\ntamaño X val', X_val.shape)
print('\ntamaño y val', y_val.shape)
print('\ntamaño X test', X_test.shape)
print('\ntamaño y test', y_test.shape)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Definir dimension de entrada para el encoder
encoder_inputs = Input(shape=(1, 4))

# Capa LSTM en el encoder
encoder_lstm = LSTM(256, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)

# Se descartan las salidas del encoder y solo se toman los estados
encoder_states = [state_h, state_c]

In [ ]:
# Definir dimension de entrada para el decoder
decoder_inputs = Input(shape=(None,))

# Capa de embedding en el decoder
decoder_embedding = Embedding(num_decoder_tokens, output_dim=100)
decoder_inputs_embedded = decoder_embedding(decoder_inputs)

# Capa LSTM en el decoder, con los estados del encoder como inicialización
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_embedded, initial_state=encoder_states)

# Capa densa con activación softmax en el output
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Definir modelo encoder-decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar el modelo
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
type(y_train)

In [ ]:
# Entrenar el modelo
model.fit([X_train.reshape(5999,1,4), y_train], y_train.reshape(y_train.shape[0], y_train.shape[1], 1), batch_size=128, epochs=15, validation_data=([X_val.reshape(X_val.shape[0],1,4), y_val], y_val.reshape(y_val.shape[0], y_val.shape[1], 1)))

In [ ]:
y_test[0]

In [ ]:
model.evaluate([X_test.reshape(X_test.shape[0],1,4),y_test[:,:-1]])

In [ ]:
import numpy as np

In [ ]:
preds = model.predict([X_test.reshape(X_test.shape[0], 1, 4), y_test[:,:-1]])

# Convertir las predicciones de índices a palabras
preds = np.argmax(preds, axis=-1)


In [ ]:
preds

In [ ]:

def predictions(preds):
    preds_words = []
    for i in range(preds.shape[0]):
        preds_row = preds[i]
        preds_row_words = []
        for idx in preds_row:
            if idx in tokenizer.index_word:
                preds_row_words.append(tokenizer.index_word[idx])
        preds_words.append(preds_row_words)

    # Imprimir las primeras 10 predicciones
    for i in range(10):
        print(preds_words[i])

    return preds_words

In [ ]:
frasesnlg = predictions(preds)

In [ ]:
x_values = scaler.inverse_transform(X_test)

In [ ]:
x_values[0]

In [ ]:
preds_words = []
for i in range(y_test.shape[0]):
    preds_row = y_test[i]
    preds_row_words = []
    for idx in preds_row:
        if idx in tokenizer.index_word:
            preds_row_words.append(tokenizer.index_word[idx])
    preds_words.append(preds_row_words)
for i in range(10):
    print(preds_words[i])

In [ ]:
pdlr = np.array([[-0.04182976,0.393413729,-3.725535952,54.77127996]])

In [ ]:
pdlrsalida =np.zeros(1,41)

In [ ]:
y_test.shape

In [ ]:
prueba = model.predict([pdlr.reshape(1,1,4),np.zeros((1,41))])

In [ ]:
prueba = np.argmax(prueba, axis=-1)

In [ ]:
prueba

In [ ]:
predictions(prueba)